# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [2]:
import pandas as pd

In [3]:
housing = pd.read_csv('cal_housing_clean.csv')

In [4]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [5]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [6]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [7]:
y_val = housing['medianHouseValue']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [14]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [15]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [16]:
import tensorflow as tf

In [17]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [18]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [36]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [37]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpwqt83mnj', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [38]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpwqt83mnj/model.ckpt.
INFO:tensorflow:loss = 348320370000.0, step = 1
INFO:tensorflow:global_step/sec: 344.035
INFO:tensorflow:loss = 1019054000000.0, step = 101 (0.301 sec)
INFO:tensorflow:global_step/sec: 339.443
INFO:tensorflow:loss = 614059200000.0, step = 201 (0.285 sec)
INFO:tensorflow:global_step/sec: 394.357
INFO:tensorflow:loss = 270141440000.0, step = 301 (0.254 sec)
INFO:tensorflow:global_step/sec: 410.343
INFO:tensorflow:loss = 291402380000.0, step = 401 (0.244 sec)
INFO:tensorflow:global_step/sec: 388.256
INFO:tensorflow:loss = 293964120000.0, step = 501 (0.259 sec)
INFO:tensorflow:global_step/sec: 376.485
INFO:tensorflow:loss = 566631660000.0, step = 601 (0.264 sec)
INFO:tensorflow:global_step/sec: 331.984
INFO:tensorflow:loss = 238454200000.0, step = 701 (0.301 sec)
INFO:tensorflow:global_step/sec: 411.526
INFO:tensorflow:loss = 309

INFO:tensorflow:loss = 84022350000.0, step = 7801 (0.436 sec)
INFO:tensorflow:global_step/sec: 374.404
INFO:tensorflow:loss = 56823520000.0, step = 7901 (0.268 sec)
INFO:tensorflow:global_step/sec: 299.043
INFO:tensorflow:loss = 62902100000.0, step = 8001 (0.329 sec)
INFO:tensorflow:global_step/sec: 288.38
INFO:tensorflow:loss = 123527640000.0, step = 8101 (0.345 sec)
INFO:tensorflow:global_step/sec: 231.975
INFO:tensorflow:loss = 42802684000.0, step = 8201 (0.441 sec)
INFO:tensorflow:global_step/sec: 322.74
INFO:tensorflow:loss = 64238174000.0, step = 8301 (0.300 sec)
INFO:tensorflow:global_step/sec: 267.935
INFO:tensorflow:loss = 32411544000.0, step = 8401 (0.382 sec)
INFO:tensorflow:global_step/sec: 278.428
INFO:tensorflow:loss = 76177860000.0, step = 8501 (0.351 sec)
INFO:tensorflow:global_step/sec: 289.579
INFO:tensorflow:loss = 183415980000.0, step = 8601 (0.349 sec)
INFO:tensorflow:global_step/sec: 364.211
INFO:tensorflow:loss = 86133195000.0, step = 8701 (0.276 sec)
INFO:tensor

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [39]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [40]:
pred_gen = model.predict(predict_input_func)

In [41]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpwqt83mnj/model.ckpt-10000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [42]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
mean_squared_error(y_test,final_preds)**0.5

102558.20074883055

# Great Job!